In [13]:
torch_ckpt = "/mnt/bn/search-douyin-rank-yg/all_data_from_lf/peitian_data/data/outputs/qwen_0.5b-listwise-d64_2layer-v4-g2-fullparam-intra/checkpoint-26793"
hf_ckpt = torch_ckpt + "-hf"
onnx_ckpt = hf_ckpt + "-onnx"

In [ ]:
# migrate from torch model to qwen model

import os
import shutil
import json

if os.path.exists(hf_ckpt):
    shutil.rmtree(hf_ckpt)

shutil.copytree(torch_ckpt, hf_ckpt)

with \
    open(os.path.join(torch_ckpt, "config.json"), "r") as f, \
    open(os.path.join(torch_ckpt, "model_args.json"), "r") as g, \
    open(os.path.join(hf_ckpt, "config.json"), "w") as h:
    config = json.load(f)
    model_args = json.load(g)
    config["mrl_dims"] = model_args["mrl_dims"]
    config["mrl_2layer_proj"] = model_args["mrl_2layer_proj"]
    config["auto_map"] = {
        "AutoConfig": "configuration_qwen2.Qwen2Config",
        "AutoModel": "modeling_qwen2.Qwen2EmbeddingModel",
        "AutoModelForSequenceClassification": "modeling_qwen2.Qwen2EmbeddingModel",
    }
    json.dump(config, h)

os.remove(os.path.join(hf_ckpt, "model_args.json"))

!cp /opt/tiger/DouyinSearchEmb/src/qwen2/* {hf_ckpt}/

In [ ]:
from optimum.exporters.onnx import main_export

from transformers import AutoConfig

from optimum.exporters.onnx.config import TextDecoderOnnxConfig, TextDecoderWithPositionIdsOnnxConfig
from optimum.utils import NormalizedTextConfig, DummyPastKeyValuesGenerator
from typing import Dict


class CustomMPTOnnxConfig(TextDecoderWithPositionIdsOnnxConfig):
    pass

config = AutoConfig.from_pretrained(hf_ckpt, trust_remote_code=True)

main_export(
    hf_ckpt,
    output=onnx_ckpt,
    task="text-classification",
    trust_remote_code=True,
    no_post_process=True,
)

!cp {hf_ckpt}/modeling_qwen2.py {onnx_ckpt}/

In [15]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_kwargs = {
    # 用FA2
    "attn_implementation": "flash_attention_2",
    # 用GPU
    "device_map": {"": "cuda"},
    # 用fp16
    "torch_dtype": torch.float16,
    # 用自定义实现
    "trust_remote_code": True
}
model = AutoModelForSequenceClassification.from_pretrained(hf_ckpt, **model_kwargs)

In [16]:
from optimum.onnxruntime import ORTModelForSequenceClassification


model_kwargs = {
    # 用FA2
    "attn_implementation": "flash_attention_2",
    # 用GPU
    "device_map": {"": "cuda"},
    # 用fp16
    "torch_dtype": torch.float16,
    # 用自定义实现
    "trust_remote_code": True
}
model_onnx = ORTModelForSequenceClassification.from_pretrained(onnx_ckpt, **model_kwargs)

In [17]:
# 按peitian代码加载

import os
import sys
if ".." not in sys.path:
    sys.path.append("..")
    os.chdir("..")

from src import ModelArgs, get_model_and_tokenizer

args = ModelArgs()

model_ref, _ = get_model_and_tokenizer(
    args, 
    model_name_or_path=torch_ckpt,
    mrl_dims=[64],
    mrl_2layer_proj=True,
    device="cuda"

    # packing is enabled by default
)

2024-12-04 15:43:48.388 | INFO     | src:get_model_and_tokenizer:36 - Loading model and tokenizer from /mnt/bn/search-douyin-rank-yg/all_data_from_lf/peitian_data/data/outputs/qwen_0.5b-listwise-d64_2layer-v4-g2-fullparam-intra/checkpoint-26793...
Some weights of the model checkpoint at /mnt/bn/search-douyin-rank-yg/all_data_from_lf/peitian_data/data/outputs/qwen_0.5b-listwise-d64_2layer-v4-g2-fullparam-intra/checkpoint-26793 were not used when initializing Qwen2Model: ['mrl_proj_64.0.bias', 'mrl_proj_64.0.weight', 'mrl_proj_64.2.bias', 'mrl_proj_64.2.weight']
- This IS expected if you are initializing Qwen2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Qwen2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceC

In [21]:
# 测试一致性

def apply_template(text, tokenizer, max_length: int = 512, is_query: bool = True):
    # 25 tokens reserved for template
    max_length = max_length - 25
    input_ids = tokenizer.encode(text, max_length=max_length, truncation=True)
    text = tokenizer.decode(input_ids, skip_special_tokens=True)

    if is_query:
        text = "Query: {text}\nUse one word to summarize the query's relevant information. The word is: \"".format(text=text)
    else:
        text = "Text: {text}\nUse one word to summarize the text's content. The word is: \"".format(text=text)
    return text

# NOTE: 左padding从而-1能直接拿到最后一个token的embedding
tokenizer = AutoTokenizer.from_pretrained(torch_ckpt, padding_side="left")

inputs1 = "I love bytedance!" * 1000
inputs1 = apply_template(inputs1, tokenizer, max_length=512)

inputs2 = "I love huawei" * 100
inputs2 = apply_template(inputs2, tokenizer, max_length=512)

device = model.device

# # padding
# inputs = tokenizer([inputs1, inputs2], padding=True, return_tensors="pt").to(device)

# # packing
input_ids = tokenizer([inputs1, inputs2]).input_ids
position_ids = [list(range(len(x))) for x in input_ids]
inputs = {
    "input_ids": torch.tensor([sum(input_ids, [])], device=device), 
    "position_ids": torch.tensor([sum(position_ids, [])], device=device)
}
inputs["attention_mask"] = torch.ones_like(inputs["input_ids"])

print(inputs.keys())

a = model_ref._encode(inputs)
b = model(**inputs)
c = model_onnx(**inputs)
# print(a == b)

dict_keys(['input_ids', 'position_ids', 'attention_mask'])
input_ids
attention_mask


In [22]:
b

tensor([[ 0.0667, -0.0531, -0.1608, -0.0148, -0.0320, -0.1418, -0.1052, -0.1243,
         -0.0372,  0.0784, -0.0833,  0.2015,  0.2404,  0.0034, -0.0937, -0.1614,
          0.0964,  0.0802, -0.0300, -0.1278,  0.1654, -0.0588, -0.0970,  0.0623,
         -0.1439, -0.0311,  0.0037,  0.0720, -0.1389, -0.0130, -0.0616, -0.0682,
          0.0182, -0.0774, -0.0529, -0.0476,  0.1059, -0.0238,  0.2202, -0.0981,
         -0.1415,  0.0725, -0.0344, -0.1008, -0.1134,  0.2151,  0.1602, -0.3391,
          0.0397,  0.0118,  0.0740, -0.0913,  0.0558, -0.0856, -0.0269, -0.0374,
         -0.0600, -0.2603, -0.2751,  0.0614, -0.0961,  0.1792, -0.2952,  0.1208],
        [ 0.1070,  0.0547, -0.1550, -0.0118, -0.0449,  0.0184, -0.0157, -0.1261,
         -0.0185,  0.0337, -0.0860,  0.0286,  0.1305,  0.0286, -0.0914, -0.2147,
          0.0804,  0.1155, -0.0446, -0.1075,  0.1941, -0.1321,  0.0490,  0.1245,
         -0.1954,  0.1311,  0.0848,  0.0395, -0.1040, -0.0903, -0.0904, -0.0871,
          0.1364, -0.2253, 

In [23]:
c

SequenceClassifierOutput(loss=None, logits=tensor([[ 0.0725, -0.0475, -0.1783, -0.0060, -0.0354, -0.1263, -0.0930, -0.1269,
         -0.0418,  0.0695, -0.0766,  0.1929,  0.2344,  0.0046, -0.0915, -0.1527,
          0.0817,  0.0846, -0.0281, -0.1302,  0.1672, -0.0646, -0.0837,  0.0728,
         -0.1544, -0.0188,  0.0141,  0.0731, -0.1461, -0.0398, -0.0716, -0.0657,
          0.0338, -0.0919, -0.0243, -0.0525,  0.1215, -0.0153,  0.2099, -0.0894,
         -0.1436,  0.0649, -0.0260, -0.0973, -0.1167,  0.1990,  0.1552, -0.3393,
          0.0549,  0.0178,  0.0767, -0.1055,  0.0516, -0.0749, -0.0268, -0.0396,
         -0.0696, -0.2497, -0.2846,  0.0709, -0.1174,  0.1789, -0.2989,  0.1329]]), hidden_states=None, attentions=None)